#IMPORTING LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input,Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

#UNZIP THE DATASET

In [ ]:
!unzip '/content/archive (1).zip'

Archive:  /content/archive (1).zip
replace spam.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

#READ THE DATASET

In [ ]:
#the provided DataSet has some error called UnicodeDecodeError
df = pd.read_csv('/content/spam.csv',delimiter=',',encoding='latin-1')

#DISPLAYING

In [ ]:
#Display the DataSet
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [ ]:
#Display the head values
df.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [ ]:
#Knowing information about the dataset like DType
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


# TAKING IMPORTANT COLUMN

In [ ]:
df.drop(['Unnamed: 2','Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)

In [ ]:
df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [ ]:
# Convert shape
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

#TEST AND TRAIN

In [ ]:
# Test and train spilit 
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [ ]:
# Tokenisation function
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

#CREATING A MODEL

In [ ]:
#creating LSTM model
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

In [ ]:
#model
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                           

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=15,validation_split=0.2)

Epoch 1/15
30/30 [==============================] - 9s 283ms/step - loss: 0.0212 - accuracy: 0.9931 - val_loss: 0.0517 - val_accuracy: 0.9873
Epoch 2/15
30/30 [==============================] - 9s 289ms/step - loss: 0.0141 - accuracy: 0.9963 - val_loss: 0.0600 - val_accuracy: 0.9863
Epoch 3/15
30/30 [==============================] - 8s 282ms/step - loss: 0.0090 - accuracy: 0.9971 - val_loss: 0.0621 - val_accuracy: 0.9863
Epoch 4/15
30/30 [==============================] - 9s 284ms/step - loss: 0.0064 - accuracy: 0.9979 - val_loss: 0.0984 - val_accuracy: 0.9852
Epoch 5/15
30/30 [==============================] - 9s 289ms/step - loss: 0.0038 - accuracy: 0.9987 - val_loss: 0.0830 - val_accuracy: 0.9842
Epoch 6/15
30/30 [==============================] - 10s 351ms/step - loss: 0.0026 - accuracy: 0.9995 - val_loss: 0.0913 - val_accuracy: 0.9873
Epoch 7/15
30/30 [==============================] - 9s 288ms/step - loss: 0.0045 - accuracy: 0.9992 - val_loss: 0.0934 - val_accuracy: 0.9863
Epoch

#CHECKING ACCURACY

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix =sequence.pad_sequences(test_sequences,maxlen=max_len)
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 28ms/step - loss: 0.1910 - accuracy: 0.9892
Accuracy: 0.989


#SAVE THE MODEL

In [ ]:
model.save("LSTM1")